# Importing Libraries

In [395]:
import pandas as pd
import datetime

from varname import nameof
import time

import numpy as np

# Functions for Calculating Technical Indicators

In [396]:
#function to get the stock name

def variable_to_str(df): 
    variable_name = list(df.keys())[1] 
    return variable_name[6:]

In [397]:
variable_to_str(GRAE)

'GRAE'

In [398]:
#Moving Average Convergence Divergence (MACD)

#https://aleksandarhaber.com/macd-moving-average-convergence-divergence-of-stock-price-time-series-in-pandas-and-python/


def MACD(data):
    
    short_period = 12
    long_period  = 26
    signal_period = 9

    stock_name = variable_to_str(df=data)
    
    data["ewm_short"] = data["Price_"+stock_name].ewm(span=short_period, adjust=False).mean()
    data["ewm_long"] = data["Price_"+stock_name].ewm(span=long_period, adjust=False).mean()
    data["MACD"] = data["ewm_short"] - data["ewm_long"]
    data["signal_MACD"] = data["MACD"].ewm(span=signal_period, adjust=False).mean()
    data["bars"] = data["MACD"] - data["signal_MACD"]
    
    data.drop(["ewm_short","ewm_long","signal_MACD","bars"],axis=1,inplace=True)
    
    data.rename(columns={"MACD":"MACD_"+stock_name},inplace=True)
    
    

In [399]:
#Relative Strength Index (RSI)

#https://www.youtube.com/watch?v=2H2Gd7zYVA8

def RSI(data):
    
    stock_name = variable_to_str(df=data)
    
    data["delta"] = data["Price_"+stock_name].diff()
    data["up"] = data["delta"].clip(lower=0)
    data["down"] = -1 * data["delta"].clip(upper=0)
    
    data["ema_up"] = data["up"].ewm(com=13, adjust=False).mean()
    data["ema_down"] = data["down"].ewm(com=13, adjust=False).mean()
    
    data["rs"] = data["ema_up"] / data["ema_down"]
    data["RSI"] = 100 - (100 / (1 + data["rs"]))
    
    data.drop(["delta","up","down","ema_up","ema_down","rs"], axis=1, inplace=True)
    data.rename(columns = {"RSI":"RSI_"+stock_name}, inplace=True)
    

In [400]:
#Commodity Channel Index (CCI)

#https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/cci
#https://gist.github.com/quantra-go-algo/1b37bfb74d69148f0dfbdb5a2c7bdb25

#TP = Typical Price
#sma = simple moving average
#mad = mean deviation

def CCI(data, number_of_days):
    
    stock_name = variable_to_str(df=data)
    
    data["TP"] = (data["High_"+stock_name] + data["Low_"+stock_name] + data["Price_"+stock_name])/3
    data['sma'] = data['TP'].rolling(number_of_days).mean()
    data['mad'] = data['TP'].rolling(number_of_days).apply(lambda x: pd.Series(x).mad())
    data['CCI'] = (data['TP'] - data['sma']) / (0.015 * data['mad']) 
    
    data.drop(["TP","sma", "mad"], axis=1, inplace=True)
    data.rename(columns = {"CCI":"CCI_"+stock_name}, inplace=True)


In [401]:
# Average Directional Index (ADX)

#https://medium.com/codex/algorithmic-trading-with-average-directional-index-in-python-2b5a20ecf06a

def calculateValue(df, column, index):
    previous_value = df.loc[index-1, column]
    current_value = df.loc[index, column]
    return current_value, previous_value

def calculateDM(df, index, stock_name):
    current_high, previous_high = calculateValue(df, "High_"+stock_name, index)
    current_low, previous_low = calculateValue(df, "Low_"+stock_name, index)
    
    dm_positive = current_high - previous_high
    dm_negative = current_low - previous_low
    
    if dm_positive > dm_negative:
        if dm_positive < 0:
            dm_positive = 0.00
        dm_negative = 0.00
        return dm_positive, dm_negative

    elif dm_positive < dm_negative:
        if dm_negative < 0:
            dm_negative = 0.00
        dm_positive = 0.00
        return dm_positive, dm_negative
    
    else:
        if dm_positive < 0:
            dm_positive = 0.00
        dm_negative = 0.00
        return dm_positive, dm_negative

def calculateTR(df, index, stock_name):
    current_high, previous_high = calculateValue(df, "High_"+stock_name, index)
    current_low, previous_low = calculateValue(df, "Low_"+stock_name, index)
    current_close, previous_close = calculateValue(df, "Price_"+stock_name, index)
    ranges = [current_high - current_low, abs(current_high - previous_close), abs(current_low - previous_close)]
    TR = max(ranges)
    return TR

def calculate_first_14(df, index, column):
    result = 0
    for i in range(index-13, index+1):
        result += df.loc[i, column]
    return result

def calculate_subsequent_14(df, index, column):
    return(df.loc[index-1, column+'14'] - (df.loc[index-1, column+'14']/14) + df.loc[index, column])


def calculate_first_adx(df, index):
    result = 0
    for i in range(index-13, index+1):
        result += df.loc[i, 'DX']
    return(result/14)

def calculate_adx(df, index):
    return(round(((df.loc[index-1, 'ADX']*13) + df.loc[index, 'DX'])/14, 2))

def ADX(data):
    
    stock_name = variable_to_str(df=data)
    
    for i in range(1, len(data)):
        dm_pos, dm_neg = calculateDM(data, i, stock_name)
        TR = calculateTR(data, i, stock_name)
        data.loc[i, '+DM'] = dm_pos
        data.loc[i, '-DM'] = dm_neg
        data.loc[i, 'TR'] = TR

        if data.TR.count() == 14:
            data.loc[i, 'TR14'] = calculate_first_14(data, i, 'TR')
            data.loc[i, '+DM14'] = calculate_first_14(data, i, '+DM')
            data.loc[i, '-DM14'] = calculate_first_14(data, i, '-DM')

        elif data.TR.count() >= 14:
            data.loc[i, 'TR14'] = round(calculate_subsequent_14(data, i, 'TR'),2)
            data.loc[i, '+DM14'] = round(calculate_subsequent_14(data, i, '+DM'), 2)
            data.loc[i, '-DM14'] = round(calculate_subsequent_14(data, i, '-DM'), 2)

        if 'TR14' in data.columns:
            data.loc[i, '+DI'] = round((data.loc[i, '+DM14'] / data.loc[i, 'TR14'])*100, 2)
            data.loc[i, '-DI'] = round((data.loc[i, '-DM14'] / data.loc[i, 'TR14'])*100, 2)

            data.loc[i, 'DX'] = round((abs(data.loc[i, '+DI'] - data.loc[i, '-DI'])/abs(data.loc[i, '+DI'] + data.loc[i, '-DI']) )*100 , 2)

        if 'DX' in data.columns:
            if data.DX.count() == 14:
                data.loc[i, 'ADX'] = calculate_first_adx(data, i)

            elif data.DX.count() >= 14:
                data.loc[i, 'ADX'] = calculate_adx(data, i)
    
    data.drop(["+DM","-DM","TR","TR14","+DM14","-DM14","+DI","-DI","DX"], axis=1, inplace=True)
    data.rename(columns={"ADX":"ADX_"+stock_name},inplace=True)


# GRAE - Grameenphone Ltd

In [408]:
GRAE = pd.read_csv("GRAE Historical Data.csv")
len_grae = len(GRAE)
print(len_grae)
GRAE.head()

3017


,Date,Price,Open,High,Low,Vol.,Change %
0,"Sep 04, 2022",288.7,295.0,295.8,288.0,121.84K,-0.45%
1,"Sep 01, 2022",290.0,290.0,291.0,289.0,161.03K,0.76%
2,"Aug 31, 2022",287.8,295.7,295.7,287.0,167.84K,-0.10%
3,"Aug 30, 2022",288.1,295.2,295.3,287.8,148.11K,-1.74%
4,"Aug 29, 2022",293.2,290.0,294.1,288.9,205.84K,1.52%


In [409]:
GRAE.tail()

,Date,Price,Open,High,Low,Vol.,Change %
3012,"Nov 24, 2009",174.8,170.0,177.0,168.8,1.32M,3.74%
3013,"Nov 23, 2009",168.5,164.1,170.0,164.1,1.09M,1.69%
3014,"Nov 22, 2009",165.7,167.0,168.9,163.0,872.40K,1.04%
3015,"Nov 19, 2009",164.0,158.1,167.9,152.0,1.63M,3.67%
3016,"Nov 18, 2009",158.2,170.1,170.2,158.0,2.73M,-7.54%


In [410]:
GRAE[GRAE["Date"]=="Mar 01, 2020"]

,Date,Price,Open,High,Low,Vol.,Change %
564,"Mar 01, 2020",270.5,277.4,277.5,268.1,162.06K,-1.42%


In [411]:
GRAE = GRAE[565:3017]
GRAE.reset_index(drop = True, inplace = True)
GRAE.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,"Feb 27, 2020",274.4,288.1,288.2,271.2,403.07K,-4.76%
1,"Feb 26, 2020",288.1,292.4,296.5,282.7,579.11K,-1.47%
2,"Feb 25, 2020",292.4,292.0,299.9,283.0,624.68K,-2.01%
3,"Feb 24, 2020",298.4,319.8,323.9,296.0,743.71K,-6.63%
4,"Feb 23, 2020",319.6,321.0,321.4,310.0,1.47M,8.12%


In [412]:
GRAE.tail()

,Date,Price,Open,High,Low,Vol.,Change %
2447,"Nov 24, 2009",174.8,170.0,177.0,168.8,1.32M,3.74%
2448,"Nov 23, 2009",168.5,164.1,170.0,164.1,1.09M,1.69%
2449,"Nov 22, 2009",165.7,167.0,168.9,163.0,872.40K,1.04%
2450,"Nov 19, 2009",164.0,158.1,167.9,152.0,1.63M,3.67%
2451,"Nov 18, 2009",158.2,170.1,170.2,158.0,2.73M,-7.54%


In [413]:
GRAE = GRAE.iloc[::-1]
GRAE.reset_index(drop=True, inplace=True)
GRAE.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,"Nov 18, 2009",158.2,170.1,170.2,158.0,2.73M,-7.54%
1,"Nov 19, 2009",164.0,158.1,167.9,152.0,1.63M,3.67%
2,"Nov 22, 2009",165.7,167.0,168.9,163.0,872.40K,1.04%
3,"Nov 23, 2009",168.5,164.1,170.0,164.1,1.09M,1.69%
4,"Nov 24, 2009",174.8,170.0,177.0,168.8,1.32M,3.74%


In [414]:
GRAE.tail()

,Date,Price,Open,High,Low,Vol.,Change %
2447,"Feb 23, 2020",319.6,321.0,321.4,310.0,1.47M,8.12%
2448,"Feb 24, 2020",298.4,319.8,323.9,296.0,743.71K,-6.63%
2449,"Feb 25, 2020",292.4,292.0,299.9,283.0,624.68K,-2.01%
2450,"Feb 26, 2020",288.1,292.4,296.5,282.7,579.11K,-1.47%
2451,"Feb 27, 2020",274.4,288.1,288.2,271.2,403.07K,-4.76%


In [415]:
GRAE.rename(columns={"Price":"Price_GRAE","Open":"Open_GRAE","High":"High_GRAE","Low":"Low_GRAE","Vol.":"Vol_GRAE","Change %":"Change_GRAE"},inplace=True)
GRAE.columns

Index(['Date', 'Price_GRAE', 'Open_GRAE', 'High_GRAE', 'Low_GRAE', 'Vol_GRAE',
       'Change_GRAE'],
      dtype='object')

In [416]:
GRAE.dtypes

Date            object
Price_GRAE     float64
Open_GRAE      float64
High_GRAE      float64
Low_GRAE       float64
Vol_GRAE        object
Change_GRAE     object
dtype: object

In [417]:
GRAE["Date"] = pd.to_datetime(GRAE["Date"])
GRAE.dtypes

Date           datetime64[ns]
Price_GRAE            float64
Open_GRAE             float64
High_GRAE             float64
Low_GRAE              float64
Vol_GRAE               object
Change_GRAE            object
dtype: object

In [335]:
print(GRAE.isna().sum().sum())
GRAE[GRAE.isna().any(axis=1)]

2


,Date,Price_GRAE,Open_GRAE,High_GRAE,Low_GRAE,Vol_GRAE,Change_GRAE
244,2019-02-17,389.9,389.9,389.9,389.9,NaN,0.00%
374,2018-08-05,389.4,389.4,389.4,389.4,NaN,0.00%


In [418]:
GRAE["Vol_GRAE"].bfill(limit=1, inplace=True)
print(GRAE.isna().sum().sum())
GRAE.reset_index(drop=True, inplace=True)

0


In [419]:
#Vol
GRAE["Vol_GRAE"] = GRAE["Vol_GRAE"].astype("str")
print(GRAE["Vol_GRAE"].apply(lambda x:x[-1:]).value_counts())
units = {'K':1e3, 'M':1e6}
GRAE["Vol_GRAE"] = GRAE["Vol_GRAE"].apply(lambda x:float(x[:-1])*units[x[-1:]])


#Change
GRAE["Change_GRAE"] = GRAE["Change_GRAE"].apply(lambda x:float(x[:-1]))

K    2138
M     314
Name: Vol_GRAE, dtype: int64


In [420]:
%%time

#calculating MACD
MACD(GRAE)

#calculating RSI
RSI(GRAE)

#calculating CCI
CCI(GRAE, number_of_days = 14)

#calculating RSI
ADX(GRAE)

CPU times: total: 3.02 s
Wall time: 2.99 s


In [421]:
GRAE.drop(["Open_GRAE", "High_GRAE", "Low_GRAE", "Change_GRAE"], axis=1, inplace=True)
GRAE.head()

,Date,Price_GRAE,Vol_GRAE,MACD_GRAE,RSI_GRAE,CCI_GRAE,ADX_GRAE
0,2009-11-18,158.2,2730000.0,0.000000,NaN,NaN,NaN
1,2009-11-19,164.0,1630000.0,0.462678,100.0,NaN,NaN
2,2009-11-22,165.7,872400.0,0.955515,100.0,NaN,NaN
3,2009-11-23,168.5,1090000.0,1.554114,100.0,NaN,NaN
4,2009-11-24,174.8,1320000.0,2.507954,100.0,NaN,NaN


In [422]:
GRAE.isna().sum()

Date           0
Price_GRAE     0
Vol_GRAE       0
MACD_GRAE      0
RSI_GRAE       1
CCI_GRAE      13
ADX_GRAE      27
dtype: int64

In [423]:
GRAE["RSI_GRAE"].bfill(limit=1, inplace=True)
GRAE["CCI_GRAE"].bfill(limit=13, inplace=True)
GRAE["ADX_GRAE"].bfill(limit=27, inplace=True)

In [424]:
GRAE.isna().sum()

Date          0
Price_GRAE    0
Vol_GRAE      0
MACD_GRAE     0
RSI_GRAE      0
CCI_GRAE      0
ADX_GRAE      0
dtype: int64

In [425]:
GRAE.dtypes

Date          datetime64[ns]
Price_GRAE           float64
Vol_GRAE             float64
MACD_GRAE            float64
RSI_GRAE             float64
CCI_GRAE             float64
ADX_GRAE             float64
dtype: object

In [426]:
GRAE.describe()

,Price_GRAE,Vol_GRAE,MACD_GRAE,RSI_GRAE,CCI_GRAE,ADX_GRAE
count,2452.000000,2.452000e+03,2452.000000,2452.000000,2452.000000,2452.000000
mean,280.038214,5.206293e+05,0.323784,51.004612,-0.387873,19.652035
std,89.981495,6.207642e+05,6.605172,14.156655,111.220294,9.636727
min,139.100000,1.889000e+04,-21.608264,8.093966,-416.427932,6.070000
25%,201.875000,1.590150e+05,-3.473460,41.354043,-85.492691,12.657500
50%,270.300000,3.236000e+05,-0.045508,49.440537,-6.867862,17.620000
75%,344.225000,6.312475e+05,3.480348,60.390015,80.521673,23.972500
max,510.700000,7.490000e+06,36.922365,100.000000,416.470588,58.660000


# SQPH - Square Pharmaceuticals Ltd 

In [427]:
SQPH = pd.read_csv("SQPH Historical Data.csv")
len_sqph = len(SQPH)
print(len_sqph)
SQPH.head()

3504


,Date,Price,Open,High,Low,Vol.,Change %
0,"Sep 04, 2022",212.6,211.8,213.5,211.5,276.44K,0.38%
1,"Sep 01, 2022",211.8,213.0,213.0,211.4,1.02M,0.33%
2,"Aug 31, 2022",211.1,212.0,213.0,209.9,892.16K,-0.38%
3,"Aug 30, 2022",211.9,213.0,213.0,211.5,679.86K,-0.09%
4,"Aug 29, 2022",212.1,214.0,214.0,211.5,764.64K,-0.52%


In [429]:
SQPH.tail()

,Date,Price,Open,High,Low,Vol.,Change %
3499,"Dec 02, 2007",46.04,47.69,47.69,45.30,1.13M,-3.01%
3500,"Nov 29, 2007",47.47,47.40,47.63,47.30,794.85K,0.02%
3501,"Nov 28, 2007",47.46,47.69,47.76,47.35,819.61K,-0.52%
3502,"Nov 27, 2007",47.71,48.76,48.76,47.68,795.76K,-1.55%
3503,"Nov 26, 2007",48.46,48.19,49.19,48.05,598.48K,1.08%


In [430]:
SQPH[SQPH["Date"]=="Mar 01, 2020"]

,Date,Price,Open,High,Low,Vol.,Change %
568,"Mar 01, 2020",184.19,184.1,185.43,176.67,610.07K,0.00%


In [431]:
SQPH = SQPH[569:3504]
SQPH.reset_index(drop = True, inplace = True)
SQPH.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,"Feb 27, 2020",184.19,189.52,189.52,183.81,481.03K,-1.88%
1,"Feb 26, 2020",187.71,190.76,191.43,187.62,597.03K,-1.50%
2,"Feb 25, 2020",190.57,191.52,194.19,190.48,719.15K,-1.67%
3,"Feb 24, 2020",193.81,195.52,196.19,192.86,390.64K,-1.02%
4,"Feb 23, 2020",195.81,198.95,199.33,194.95,247.44K,-1.11%


In [432]:
SQPH.tail()

,Date,Price,Open,High,Low,Vol.,Change %
2930,"Dec 02, 2007",46.04,47.69,47.69,45.30,1.13M,-3.01%
2931,"Nov 29, 2007",47.47,47.40,47.63,47.30,794.85K,0.02%
2932,"Nov 28, 2007",47.46,47.69,47.76,47.35,819.61K,-0.52%
2933,"Nov 27, 2007",47.71,48.76,48.76,47.68,795.76K,-1.55%
2934,"Nov 26, 2007",48.46,48.19,49.19,48.05,598.48K,1.08%


In [433]:
SQPH = SQPH.iloc[::-1]
SQPH.reset_index(drop=True, inplace=True)
SQPH.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,"Nov 26, 2007",48.46,48.19,49.19,48.05,598.48K,1.08%
1,"Nov 27, 2007",47.71,48.76,48.76,47.68,795.76K,-1.55%
2,"Nov 28, 2007",47.46,47.69,47.76,47.35,819.61K,-0.52%
3,"Nov 29, 2007",47.47,47.40,47.63,47.30,794.85K,0.02%
4,"Dec 02, 2007",46.04,47.69,47.69,45.30,1.13M,-3.01%


In [434]:
SQPH.tail()

,Date,Price,Open,High,Low,Vol.,Change %
2930,"Feb 23, 2020",195.81,198.95,199.33,194.95,247.44K,-1.11%
2931,"Feb 24, 2020",193.81,195.52,196.19,192.86,390.64K,-1.02%
2932,"Feb 25, 2020",190.57,191.52,194.19,190.48,719.15K,-1.67%
2933,"Feb 26, 2020",187.71,190.76,191.43,187.62,597.03K,-1.50%
2934,"Feb 27, 2020",184.19,189.52,189.52,183.81,481.03K,-1.88%


In [435]:
SQPH.rename(columns={"Price":"Price_SQPH","Open":"Open_SQPH","High":"High_SQPH","Low":"Low_SQPH","Vol.":"Vol_SQPH","Change %":"Change_SQPH"},inplace=True)
SQPH.columns

Index(['Date', 'Price_SQPH', 'Open_SQPH', 'High_SQPH', 'Low_SQPH', 'Vol_SQPH',
       'Change_SQPH'],
      dtype='object')

In [436]:
SQPH.dtypes

Date            object
Price_SQPH     float64
Open_SQPH      float64
High_SQPH      float64
Low_SQPH       float64
Vol_SQPH        object
Change_SQPH     object
dtype: object

In [437]:
SQPH["Date"] = pd.to_datetime(SQPH["Date"])
SQPH.dtypes

Date           datetime64[ns]
Price_SQPH            float64
Open_SQPH             float64
High_SQPH             float64
Low_SQPH              float64
Vol_SQPH               object
Change_SQPH            object
dtype: object

In [438]:
print(SQPH.isna().sum().sum())
SQPH[SQPH.isna().any(axis=1)]

1


,Date,Price_SQPH,Open_SQPH,High_SQPH,Low_SQPH,Vol_SQPH,Change_SQPH
86,2008-04-06,62.18,62.18,62.18,62.18,NaN,0.00%


In [439]:
SQPH["Vol_SQPH"].bfill(limit=1, inplace=True)
print(SQPH.isna().sum().sum())
SQPH.reset_index(drop=True, inplace=True)

0


In [440]:
#Vol
SQPH["Vol_SQPH"] = SQPH["Vol_SQPH"].astype("str")
SQPH["Vol_SQPH"].apply(lambda x:x[-1:]).value_counts()
units = {'K':1e3, 'M':1e6}
SQPH["Vol_SQPH"] = SQPH["Vol_SQPH"].apply(lambda x:float(x[:-1])*units[x[-1:]])

#Change
SQPH["Change_SQPH"] = SQPH["Change_SQPH"].apply(lambda x:float(x[:-1]))

In [441]:
%%time

#calculating MACD
MACD(SQPH)

#calculating RSI
RSI(SQPH)

#calculating CCI
CCI(SQPH, number_of_days = 14)

#calculating RSI
ADX(SQPH)

CPU times: total: 3.58 s
Wall time: 3.6 s


In [442]:
SQPH.drop(["Open_SQPH", "High_SQPH", "Low_SQPH", "Change_SQPH"], axis=1, inplace=True)
SQPH.head()

,Date,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH
0,2007-11-26,48.46,598480.0,0.000000,NaN,NaN,NaN
1,2007-11-27,47.71,795760.0,-0.059829,0.000000,NaN,NaN
2,2007-11-28,47.46,819610.0,-0.125965,0.000000,NaN,NaN
3,2007-11-29,47.47,794850.0,-0.175547,0.107576,NaN,NaN
4,2007-12-02,46.04,1130000.0,-0.326468,0.092287,NaN,NaN


In [443]:
SQPH.isna().sum()

Date           0
Price_SQPH     0
Vol_SQPH       0
MACD_SQPH      0
RSI_SQPH       1
CCI_SQPH      13
ADX_SQPH      27
dtype: int64

In [444]:
SQPH["RSI_SQPH"].bfill(limit=1, inplace=True)
SQPH["CCI_SQPH"].bfill(limit=13, inplace=True)
SQPH["ADX_SQPH"].bfill(limit=27, inplace=True)

In [445]:
SQPH.isna().sum()

Date          0
Price_SQPH    0
Vol_SQPH      0
MACD_SQPH     0
RSI_SQPH      0
CCI_SQPH      0
ADX_SQPH      0
dtype: int64

In [446]:
SQPH.head()

,Date,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH
0,2007-11-26,48.46,598480.0,0.000000,0.000000,-36.783439,27.256429
1,2007-11-27,47.71,795760.0,-0.059829,0.000000,-36.783439,27.256429
2,2007-11-28,47.46,819610.0,-0.125965,0.000000,-36.783439,27.256429
3,2007-11-29,47.47,794850.0,-0.175547,0.107576,-36.783439,27.256429
4,2007-12-02,46.04,1130000.0,-0.326468,0.092287,-36.783439,27.256429


In [451]:
SQPH.tail()

,Date,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH
2930,2020-02-23,195.81,247440.0,3.812574,63.883782,87.185340,19.49
2931,2020-02-24,193.81,390640.0,3.588574,59.000165,37.956453,19.46
2932,2020-02-25,190.57,719150.0,3.113719,52.057391,-11.144639,19.43
2933,2020-02-26,187.71,597030.0,2.478049,46.819979,-66.086286,19.40
2934,2020-02-27,184.19,481030.0,1.670980,41.311109,-119.782695,19.38


In [447]:
SQPH.dtypes

Date          datetime64[ns]
Price_SQPH           float64
Vol_SQPH             float64
MACD_SQPH            float64
RSI_SQPH             float64
CCI_SQPH             float64
ADX_SQPH             float64
dtype: object

In [448]:
SQPH.describe()

,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH
count,2935.000000,2.935000e+03,2935.000000,2935.000000,2935.000000,2935.000000
mean,138.003141,1.046896e+06,0.332374,52.494938,9.868091,18.344041
std,73.828892,1.122216e+06,2.087979,14.049716,112.804252,8.192484
min,39.450000,7.611000e+04,-8.641051,0.000000,-392.036043,5.240000
25%,73.760000,3.729650e+05,-0.609138,43.676759,-78.286290,11.945000
50%,134.620000,6.581500e+05,0.260326,52.226239,11.525502,16.700000
75%,200.020000,1.300000e+06,1.273386,61.102009,89.029550,23.030000
max,291.550000,1.288000e+07,8.363396,90.632171,406.812418,54.430000


# ACIF - ACI Formulations Ltd

In [452]:
ACIF = pd.read_csv("ACIF Historical Data.csv")
len_acif = len(ACIF)
print(len_acif)
ACIF.head()

3268


,Date,Price,Open,High,Low,Vol.,Change %
0,"Sep 04, 2022",173.8,183.0,184.0,171.6,831.80K,-0.57%
1,"Sep 01, 2022",174.8,174.4,175.8,170.9,551.36K,1.92%
2,"Aug 31, 2022",171.5,166.5,174.9,163.1,1.16M,4.38%
3,"Aug 30, 2022",164.3,163.0,166.2,162.8,304.96K,1.05%
4,"Aug 29, 2022",162.6,165.0,166.0,161.5,387.64K,-1.28%


In [453]:
ACIF.tail()

,Date,Price,Open,High,Low,Vol.,Change %
3263,"Nov 27, 2008",82.22,84.66,92.59,80.69,34.40K,-8.05%
3264,"Nov 26, 2008",89.42,91.64,91.64,82.01,4.54K,-6.27%
3265,"Nov 25, 2008",95.40,100.53,100.53,94.71,6.61K,-4.35%
3266,"Nov 24, 2008",99.74,95.24,105.29,95.24,7.37K,-0.88%
3267,"Nov 23, 2008",100.63,106.98,115.87,100.53,37.42K,-5.94%


In [454]:
ACIF[ACIF["Date"]=="Mar 01, 2020"]

,Date,Price,Open,High,Low,Vol.,Change %
568,"Mar 01, 2020",91.05,93.52,93.62,88.67,11.35K,-3.04%


In [455]:
ACIF = ACIF[569:3268]
ACIF.reset_index(drop = True, inplace = True)
ACIF.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,"Feb 27, 2020",93.90,95.14,96.19,93.43,6.56K,-1.20%
1,"Feb 26, 2020",95.05,97.14,97.14,94.76,16.62K,-1.29%
2,"Feb 25, 2020",96.29,97.43,99.81,96.10,16.37K,-0.79%
3,"Feb 24, 2020",97.05,98.38,100.76,96.29,11.92K,-1.16%
4,"Feb 23, 2020",98.19,104.48,104.48,98.10,7.86K,-1.72%


In [456]:
ACIF.tail()

,Date,Price,Open,High,Low,Vol.,Change %
2694,"Nov 27, 2008",82.22,84.66,92.59,80.69,34.40K,-8.05%
2695,"Nov 26, 2008",89.42,91.64,91.64,82.01,4.54K,-6.27%
2696,"Nov 25, 2008",95.40,100.53,100.53,94.71,6.61K,-4.35%
2697,"Nov 24, 2008",99.74,95.24,105.29,95.24,7.37K,-0.88%
2698,"Nov 23, 2008",100.63,106.98,115.87,100.53,37.42K,-5.94%


In [457]:
ACIF = ACIF.iloc[::-1]
ACIF.reset_index(drop=True, inplace=True)
ACIF.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,"Nov 23, 2008",100.63,106.98,115.87,100.53,37.42K,-5.94%
1,"Nov 24, 2008",99.74,95.24,105.29,95.24,7.37K,-0.88%
2,"Nov 25, 2008",95.40,100.53,100.53,94.71,6.61K,-4.35%
3,"Nov 26, 2008",89.42,91.64,91.64,82.01,4.54K,-6.27%
4,"Nov 27, 2008",82.22,84.66,92.59,80.69,34.40K,-8.05%


In [458]:
ACIF.tail()

,Date,Price,Open,High,Low,Vol.,Change %
2694,"Feb 23, 2020",98.19,104.48,104.48,98.10,7.86K,-1.72%
2695,"Feb 24, 2020",97.05,98.38,100.76,96.29,11.92K,-1.16%
2696,"Feb 25, 2020",96.29,97.43,99.81,96.10,16.37K,-0.79%
2697,"Feb 26, 2020",95.05,97.14,97.14,94.76,16.62K,-1.29%
2698,"Feb 27, 2020",93.90,95.14,96.19,93.43,6.56K,-1.20%


In [459]:
ACIF.rename(columns={"Price":"Price_ACIF","Open":"Open_ACIF","High":"High_ACIF","Low":"Low_ACIF","Vol.":"Vol_ACIF","Change %":"Change_ACIF"},inplace=True)
ACIF.columns

Index(['Date', 'Price_ACIF', 'Open_ACIF', 'High_ACIF', 'Low_ACIF', 'Vol_ACIF',
       'Change_ACIF'],
      dtype='object')

In [460]:
ACIF.dtypes

Date            object
Price_ACIF     float64
Open_ACIF      float64
High_ACIF      float64
Low_ACIF       float64
Vol_ACIF        object
Change_ACIF     object
dtype: object

In [461]:
ACIF["Date"] = pd.to_datetime(ACIF["Date"])
ACIF.dtypes

Date           datetime64[ns]
Price_ACIF            float64
Open_ACIF             float64
High_ACIF             float64
Low_ACIF              float64
Vol_ACIF               object
Change_ACIF            object
dtype: object

In [462]:
print(ACIF.isna().sum().sum())

0


In [463]:
#Vol
ACIF["Vol_ACIF"] = ACIF["Vol_ACIF"].astype("str")
ACIF["Vol_ACIF"].apply(lambda x:x[-1:]).value_counts()
units = {'K':1e3, 'M':1e6}
ACIF["Vol_ACIF"] = ACIF["Vol_ACIF"].apply(lambda x:float(x[:-1])*units[x[-1:]])

#Change
ACIF["Change_ACIF"] = ACIF["Change_ACIF"].apply(lambda x:float(x[:-1]))

In [464]:
%%time

#calculating MACD
MACD(ACIF)

#calculating RSI
RSI(ACIF)

#calculating CCI
CCI(ACIF, number_of_days = 14)

#calculating RSI
ADX(ACIF)

CPU times: total: 3.67 s
Wall time: 3.67 s


In [465]:
ACIF.drop(["Open_ACIF", "High_ACIF", "Low_ACIF", "Change_ACIF"], axis=1, inplace=True)
ACIF.head()

,Date,Price_ACIF,Vol_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
0,2008-11-23,100.63,37420.0,0.000000,NaN,NaN,NaN
1,2008-11-24,99.74,7370.0,-0.070997,0.0,NaN,NaN
2,2008-11-25,95.40,6610.0,-0.472023,0.0,NaN,NaN
3,2008-11-26,89.42,4540.0,-1.257876,0.0,NaN,NaN
4,2008-11-27,82.22,34400.0,-2.433596,0.0,NaN,NaN


In [466]:
ACIF.isna().sum()

Date           0
Price_ACIF     0
Vol_ACIF       0
MACD_ACIF      0
RSI_ACIF       1
CCI_ACIF      13
ADX_ACIF      27
dtype: int64

In [467]:
ACIF["RSI_ACIF"].bfill(limit=1, inplace=True)
ACIF["CCI_ACIF"].bfill(limit=13, inplace=True)
ACIF["ADX_ACIF"].bfill(limit=27, inplace=True)

In [468]:
ACIF.isna().sum()

Date          0
Price_ACIF    0
Vol_ACIF      0
MACD_ACIF     0
RSI_ACIF      0
CCI_ACIF      0
ADX_ACIF      0
dtype: int64

In [469]:
ACIF.head()

,Date,Price_ACIF,Vol_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
0,2008-11-23,100.63,37420.0,0.000000,0.0,46.645149,37.230714
1,2008-11-24,99.74,7370.0,-0.070997,0.0,46.645149,37.230714
2,2008-11-25,95.40,6610.0,-0.472023,0.0,46.645149,37.230714
3,2008-11-26,89.42,4540.0,-1.257876,0.0,46.645149,37.230714
4,2008-11-27,82.22,34400.0,-2.433596,0.0,46.645149,37.230714


In [470]:
ACIF.dtypes

Date          datetime64[ns]
Price_ACIF           float64
Vol_ACIF             float64
MACD_ACIF            float64
RSI_ACIF             float64
CCI_ACIF             float64
ADX_ACIF             float64
dtype: object

In [471]:
ACIF.describe()

,Price_ACIF,Vol_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
count,2699.000000,2.699000e+03,2699.000000,2699.000000,2699.000000,2699.000000
mean,121.370574,1.713338e+05,-0.024973,48.484212,-11.268605,24.299896
std,43.478839,3.297330e+05,3.512141,11.924202,108.552677,11.262465
min,40.320000,6.900000e+02,-7.263533,0.000000,-362.554357,4.620000
25%,80.760000,2.758500e+04,-1.859532,40.768171,-94.411376,15.435000
50%,115.750000,7.308000e+04,-0.442777,47.389163,-27.014851,21.570000
75%,155.520000,1.882550e+05,1.205012,55.493691,64.940261,32.010000
max,242.860000,8.600000e+06,20.388445,95.534066,421.432330,60.160000


# Merge - 3 Stocks

In [472]:
print(min(len_grae, len_sqph, len_acif))
print(max(len_grae, len_sqph, len_acif))

3017
3504


In [473]:
data = pd.merge(pd.merge(GRAE,SQPH,on='Date'),ACIF,on='Date')
print(len(data))
data.head()

2429


,Date,Price_GRAE,Vol_GRAE,MACD_GRAE,RSI_GRAE,CCI_GRAE,ADX_GRAE,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH,Price_ACIF,Vol_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
0,2009-11-18,158.2,2730000.0,0.000000,100.0,-5.840677,14.040714,50.47,3310000.0,0.485585,63.804225,92.895442,17.86,109.71,85210.0,-2.088105,35.793253,-82.824393,43.85
1,2009-11-19,164.0,1630000.0,0.462678,100.0,-5.840677,14.040714,50.56,2540000.0,0.525514,64.444751,91.644205,16.70,109.65,108200.0,-2.195135,35.680455,-74.623541,43.29
2,2009-11-22,165.7,872400.0,0.955515,100.0,-5.840677,14.040714,50.46,2160000.0,0.542831,63.108436,81.210533,15.81,110.60,116230.0,-2.178191,38.960434,-48.865595,41.84
3,2009-11-23,168.5,1090000.0,1.554114,100.0,-5.840677,14.040714,51.80,5750000.0,0.657107,71.605138,158.707188,16.71,110.29,124580.0,-2.164822,38.274534,-43.144851,40.52
4,2009-11-24,174.8,1320000.0,2.507954,100.0,-5.840677,14.040714,52.81,3980000.0,0.819721,76.077450,225.934820,15.52,112.57,123160.0,-1.947797,45.828360,8.339818,37.73


In [474]:
data.tail()

,Date,Price_GRAE,Vol_GRAE,MACD_GRAE,RSI_GRAE,CCI_GRAE,ADX_GRAE,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH,Price_ACIF,Vol_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
2424,2020-02-23,319.6,1470000.0,5.475102,75.478818,277.962985,39.02,195.81,247440.0,3.812574,63.883782,87.185340,19.49,98.19,7860.0,3.409751,60.493740,73.682645,33.12
2425,2020-02-24,298.4,743710.0,6.731499,61.494906,151.244259,36.85,193.81,390640.0,3.588574,59.000165,37.956453,19.46,97.05,11920.0,3.286671,58.269329,44.018616,32.08
2426,2020-02-25,292.4,624680.0,7.160511,58.208007,71.993974,34.84,190.57,719150.0,3.113719,52.057391,-11.144639,19.43,96.29,16370.0,3.092159,56.770606,31.659258,31.12
2427,2020-02-26,288.1,579110.0,7.072011,55.901919,52.046668,32.97,187.71,597030.0,2.478049,46.819979,-66.086286,19.40,95.05,16620.0,2.805607,54.315890,7.520990,30.23
2428,2020-02-27,274.4,403070.0,5.829202,49.212270,4.081234,31.23,184.19,481030.0,1.670980,41.311109,-119.782695,19.38,93.90,6560.0,2.457391,52.067333,-16.153620,29.40


In [475]:
data.columns

Index(['Date', 'Price_GRAE', 'Vol_GRAE', 'MACD_GRAE', 'RSI_GRAE', 'CCI_GRAE',
       'ADX_GRAE', 'Price_SQPH', 'Vol_SQPH', 'MACD_SQPH', 'RSI_SQPH',
       'CCI_SQPH', 'ADX_SQPH', 'Price_ACIF', 'Vol_ACIF', 'MACD_ACIF',
       'RSI_ACIF', 'CCI_ACIF', 'ADX_ACIF'],
      dtype='object')

In [476]:
data.isna().sum().sum()

0

In [477]:
data.to_csv("Stock Data - Excluding COVID 19 .csv",index=False)